In [ ]:
import modulos as md
import numpy as np
import constantes as cons
import os
import sys
lista_filtrada = []

lista=md.lista_de_monedas()
#lista=md.filtradodemonedas ()
#lista=['SPELLUSDT', 'AXSUSDT', 'ETHUSDT', 'BNBUSDT', 'OGNUSDT', 'CYBERUSDT', '1000PEPEUSDT', 'COMPUSDT', 'ADAUSDT', 'BCHUSDT', 'RUNEUSDT', 'MATICUSDT', 'BLZUSDT', 'HBARUSDT', 'TRBUSDT', 'WLDUSDT', 'MTLUSDT', 'BTCUSDT', 'XRPUSDT', 'OPUSDT', 'PERPUSDT', 'ARBUSDT', 'STMXUSDT', 'CFXUSDT', 'SOLUSDT', 'DOGEUSDT', 'APEUSDT', 'UNFIUSDT', 'LTCUSDT', 'STORJUSDT', 'CRVUSDT']
#lista=['STMXUSDT']

if len(lista)>1:
    imprimo=False
    debug=False
else:
    imprimo= True
    debug=True
for symbol in lista:    
    try:
        #data,porcentajeentrada = md.estrategia_santa(symbol,tp_flag = True)
        #data,porcentajeentrada = md.estrategia_trampa(symbol,tp_flag = True)
        data = md.estrategia_haz(symbol, debug=debug)
        #data,porcentajeentrada = md.estrategia_oro('XAU',tp_flag = True)
        #######################################################################
        resultado = md.backtesting(data, plot_flag = imprimo)
        #resultado = md.backtestingsanta(data, plot_flag = imprimo, debug = True)
        ########################################################################
        #if resultado['Return [%]'] < 0: # para santa3 mazmorra
        #if data.disparo.iloc[-1]!=0: # imprime las monedas que estan en posible haz martillo
        #        print(symbol)
        if resultado['Return [%]'] > 0:
            #if ((resultado['Profit Factor'] > 2 or np.isnan(resultado['Profit Factor'])) and (resultado['Return [%]']/resultado['# Trades']) >=0.33):            
                lista_filtrada.append(symbol)
        print(f"{symbol} - Return [%]: {md.truncate(resultado['Return [%]'],2)}% - # Trades: {resultado['# Trades']} - Profit Factor: {resultado['Profit Factor']} - Win Rate [%]: {resultado['Win Rate [%]']}")
    except Exception as falla:
        _, _, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print("\nError: "+str(falla)+" - line: "+str(exc_tb.tb_lineno)+" - file: "+str(fname)+"\n")
        pass
for symbol in lista_filtrada:
    md.printandlog(cons.nombrelog,symbol,pal=1)
#md.dibuja_patrones_triangulos (data,998)    


In [42]:
import modulos as md
import numpy as np
from backtesting import Backtest
import pandas_ta as ta
from backtesting import Strategy
import pandas as pd

def backtesting(data, plot_flag=False):
    balance = 100    
    def indicador(df_campo):
        indi=pd.Series(df_campo)
        return indi.to_numpy()
    def ema(df_campo):
        indi=ta.ema(df_campo.s,length=100)
        return indi.to_numpy()   
    class Fenix(Strategy):
        def init(self):
            super().init()
            #self.pivot_high = self.I(indicador,self.data.pivot_high)
            #self.pivot_low = self.I(indicador,self.data.pivot_low)
            self.decisional = self.I(indicador,self.data.decisional)
            self.sma = self.I(ema,self.data.tendencia)
            self.tendencia = self.I(indicador,self.data.tendencia)
        def next(self):       
            super().next()
            if self.position:
                if self.data.cierra[-1]==True:
                    self.position.close()                    
            else:   
                if np.isnan(data.take_profit[-1]):
                    tp_value = None
                else:
                    tp_value = self.data.take_profit[-1]
                size= balance*self.data.porcentajeentrada[-1]/100
                if self.data.signal[-1]==1:
                    self.buy(size=size,sl=self.data.stop_loss[-1],tp=tp_value)
                elif self.data.signal[-1]==-1:
                    self.sell(size=size,sl=self.data.stop_loss[-1],tp=tp_value)
    bt = Backtest(data, Fenix, cash=balance)
    output = bt.run()
    if plot_flag:
        bt.plot()
    return output

####################################################################################

def smart_money(df):
    # Función para detectar los puntos pivote High y Low
    # Parámetros de longitud para los puntos pivote High y Low
    leftLenH    = 1
    rightLenH   = 1
    leftLenL    = 1
    rightLenL   = 1
    df['pivot_high'] = np.NaN
    df['pivot_low'] = np.NaN
    df['row_number'] = (range(len(df)))
    df.set_index('row_number', inplace=True)
    ## PIVOTS SUPERIORES
    for i in range(leftLenH, len(df) - rightLenH):
        if (
            df['High'].iloc[i] == df['High'].iloc[i - leftLenH:i + rightLenH + 1].max()
            and df['High'].iloc[i] > df['High'].iloc[i - 1]
            and df['High'].iloc[i] > df['High'].iloc[i + 1]
            ):
            df.at[i, 'pivot_high'] = df['High'].iloc[i]
    ## PIVOTS INFERIORES
    for i in range(leftLenL, len(df) - rightLenL):
        if (
            df['Low'].iloc[i] == df['Low'].iloc[i - leftLenL:i + rightLenL + 1].min()
            and df['Low'].iloc[i] < df['Low'].iloc[i - 1]
            and df['Low'].iloc[i] < df['Low'].iloc[i + 1]
            ):
            df.at[i, 'pivot_low'] = df['Low'].iloc[i]
    ## RELLANO DE PIVOTS
    for i in range(0, len(df)):
        if np.isnan(df['pivot_low'].iloc[i]):
            df.at[i, 'pivot_low'] = df['pivot_low'].iloc[i - 1]
        if np.isnan(df['pivot_high'].iloc[i]):
            df.at[i, 'pivot_high'] = df['pivot_high'].iloc[i - 1]
    ## TENDENCIA
    df['chg_high'] = np.where((df.pivot_high>df.pivot_high.shift(1))
                               ,1,
                               np.where((df.pivot_high<df.pivot_high.shift(1))                               
                               ,-1,
                               np.NaN))
    df['chg_low'] = np.where((df.pivot_low>df.pivot_low.shift(1))
                               ,1,
                               np.where((df.pivot_low<df.pivot_low.shift(1))                               
                               ,-1,
                               np.NaN))        
    for i in range(0, len(df)):
        if np.isnan(df['chg_low'].iloc[i]):
            df.at[i, 'chg_low'] = df['chg_low'].iloc[i - 1]
        if np.isnan(df['chg_high'].iloc[i]):
            df.at[i, 'chg_high'] = df['chg_high'].iloc[i - 1]
    df['tendencia'] = np.where(
                                (df.chg_high == 1) & (df.chg_low == 1)
                               ,1,
                               np.where(
                                        (df.chg_high == -1) & (df.chg_low == -1)                                                                     
                                        ,-1,
                                        0)
                                )
    #for i in range(0, len(df)):
    #    if np.isnan(df['tendencia'].iloc[i]):
    #        df.at[i, 'tendencia'] = df['tendencia'].iloc[i - 1]
    # DECISIONALES
    df['color'] = np.where(df.Close > df.Open,'verde','rojo')
    df['decisional'] = np.where(                                
                                (df.color == 'verde') # vela actual verde
                                & (df.color.shift(1) == 'rojo') # vela anterior roja
                                & (df.color.shift(-2) == 'rojo') # segunda vela roja                                
                                & (df.High.shift(-3) < df.Low)    
                                & (df.atr >= df.Close-df.Open)
                                ,df.Low,
                                np.where(
                                        (df.color == 'rojo') 
                                        & (df.color.shift(1) == 'verde') 
                                        & (df.color.shift(-2) == 'verde')
                                        & (df.Low.shift(-3) > df.High)                
                                        & (df.atr >= df.Open-df.Close)
                                        ,df.High,
                                        np.NaN)
                                )
    df['decisional_flag'] = df['decisional']
    for i in range(0, len(df)):
        if np.isnan(df['decisional'].iloc[i]):
            df.at[i, 'decisional'] = df['decisional'].iloc[i - 1]
    ## indice
    df['timestamp']=df['Open Time']
    df.set_index('timestamp', inplace=True)

##########################################################################################

def estrategia_real(symbol,debug = False):
    try:                
        np.seterr(divide='ignore', invalid='ignore')
        # temporalidad de 1h para encontrar martillos y soportes/resistencias
        data = md.obtiene_historial(symbol,'15m')      
        smart_money(data)          
        data.drop(['ema20','ema50', 'ema200'], axis=1, inplace=True)    
        data['signal'] = 0
        data['take_profit'] = np.NaN
        data['stop_loss'] = np.NaN
        data['cierra'] = False
        # Reemplazar valores no finitos (NA e inf) con 0
        data['porcentajeentrada'] = np.nan_to_num((data.Close/data.atr), nan=0, posinf=0, neginf=0)
        # Aplicar np.floor y convertir a enteros
        data['porcentajeentrada'] = np.floor(data['porcentajeentrada']).astype(int)
        ####################### alertas y valores
        if debug:
            df_str = data[list(data.columns)].to_string(index=False)
            print(df_str)
        return data
    except Exception as falla:
        _, _, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print("\nError: "+str(falla)+" - line: "+str(exc_tb.tb_lineno)+" - file: "+str(fname)+" - symbol: "+symbol+"\n")
        pass

#######################################################################################################################

symbol='STGUSDT'
data = estrategia_real(symbol)
backtesting(data, plot_flag=True)

for index, values in data.iterrows():
    if not np.isnan(values.decisional_flag):
        print(f"{values['Open Time']} | {values.Close} | {values.pivot_low} | {values.decisional_flag}")

data.tail(60)     

C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


2023-09-08 00:00:00 | 0.5404 | 0.5391 | 0.5423
2023-09-08 09:30:00 | 0.5496 | 0.544 | 0.5468
2023-09-08 13:00:00 | 0.5427 | 0.5375 | 0.5436
2023-09-08 18:00:00 | 0.5387 | 0.5383 | 0.5395
2023-09-08 22:45:00 | 0.5405 | 0.5405 | 0.5415
2023-09-09 03:30:00 | 0.5457 | 0.5442 | 0.5446
2023-09-09 21:15:00 | 0.5391 | 0.539 | 0.5402
2023-09-09 22:30:00 | 0.5399 | 0.539 | 0.5397
2023-09-09 23:30:00 | 0.5396 | 0.5387 | 0.5389
2023-09-10 11:45:00 | 0.5233 | 0.5232 | 0.5254
2023-09-10 18:45:00 | 0.5239 | 0.5228 | 0.5229
2023-09-11 05:00:00 | 0.5156 | 0.5152 | 0.5181
2023-09-11 06:45:00 | 0.5187 | 0.518 | 0.5199
2023-09-11 12:15:00 | 0.5126 | 0.5096 | 0.5131
2023-09-11 16:45:00 | 0.4875 | 0.4908 | 0.485
2023-09-11 18:15:00 | 0.4804 | 0.4786 | 0.4793
2023-09-11 21:00:00 | 0.4759 | 0.4675 | 0.4732
2023-09-12 10:30:00 | 0.479 | 0.4754 | 0.481
2023-09-13 07:45:00 | 0.4445 | 0.4425 | 0.4469
2023-09-13 13:45:00 | 0.4453 | 0.44 | 0.4467
2023-09-13 18:45:00 | 0.4453 | 0.4429 | 0.4471
2023-09-13 19:45:00 | 

,Open Time,Open,High,Low,Close,Volume,atr,pivot_high,pivot_low,chg_high,chg_low,tendencia,color,decisional,decisional_flag,signal,take_profit,stop_loss,cierra,porcentajeentrada
2023-09-17 07:30:00,2023-09-17 07:30:00,0.4566,0.4569,0.4548,0.4548,137998.0,0.001897,0.4564,0.4540,1.0,1.0,1,rojo,0.4556,NaN,0,NaN,NaN,False,239
2023-09-17 07:45:00,2023-09-17 07:45:00,0.4549,0.4549,0.4526,0.4533,181629.0,0.001926,0.4564,0.4540,1.0,1.0,1,rojo,0.4556,NaN,0,NaN,NaN,False,235
2023-09-17 08:00:00,2023-09-17 08:00:00,0.4533,0.4534,0.4506,0.4522,504047.0,0.001988,0.4564,0.4506,1.0,-1.0,0,rojo,0.4556,NaN,0,NaN,NaN,False,227
2023-09-17 08:15:00,2023-09-17 08:15:00,0.4521,0.4529,0.4514,0.4528,75319.0,0.001953,0.4564,0.4506,1.0,-1.0,0,verde,0.4556,NaN,0,NaN,NaN,False,231
2023-09-17 08:30:00,2023-09-17 08:30:00,0.4529,0.4549,0.4529,0.4546,152888.0,0.001964,0.4564,0.4506,1.0,-1.0,0,verde,0.4556,NaN,0,NaN,NaN,False,231
2023-09-17 08:45:00,2023-09-17 08:45:00,0.4546,0.4555,0.4545,0.4554,73273.0,0.001895,0.4564,0.4506,1.0,-1.0,0,verde,0.4556,NaN,0,NaN,NaN,False,240
2023-09-17 09:00:00,2023-09-17 09:00:00,0.4554,0.4558,0.4542,0.4542,157311.0,0.001874,0.4558,0.4506,-1.0,-1.0,-1,rojo,0.4556,NaN,0,NaN,NaN,False,242
2023-09-17 09:15:00,2023-09-17 09:15:00,0.4542,0.4547,0.4539,0.4544,56111.0,0.001797,0.4558,0.4506,-1.0,-1.0,-1,verde,0.4556,NaN,0,NaN,NaN,False,252
2023-09-17 09:30:00,2023-09-17 09:30:00,0.4544,0.4550,0.4539,0.4548,76441.0,0.001747,0.4558,0.4506,-1.0,-1.0,-1,verde,0.4556,NaN,0,NaN,NaN,False,260
2023-09-17 09:45:00,2023-09-17 09:45:00,0.4549,0.4560,0.4549,0.4557,103344.0,0.001708,0.4560,0.4506,1.0,-1.0,0,verde,0.4556,NaN,0,NaN,NaN,False,266
